In [ ]:
#import zipfile
import numpy as np
import pandas
import os
import io
from pprint import pprint
import json
from datetime import datetime
print('imported')

In [ ]:
# import list of teams from retrosheet
teams_df = pandas.DataFrame.from_csv('teams.csv')
teams_df['firstGame'] = pandas.to_datetime(teams_df['firstGame'], errors='coerce')
teams_df['lastGame'] = pandas.to_datetime(teams_df['lastGame'], errors='coerce')

teams_df['firstYear'] = pandas.to_numeric(teams_df['firstGame'].apply(lambda x: x.year), downcast='integer', errors='coerce')
teams_df['lastYear'] = pandas.to_numeric(teams_df['lastGame'].apply(lambda x: x.year), downcast='integer', errors='coerce')
teams_df['lastYear'] = teams_df['lastYear'].fillna(2016)

teams_df = teams_df.fillna('')
print('ok')
#teams_df

In [ ]:
allfiles = []
allgames = []
gamecount = -1  # MUST START AT -1

ladj_games = []
badj_games = []
padj_games = []

yeararray = np.linspace(1921,2016,96)
#yeararray = [2015]

for thisyear in yeararray:
    allfiles = []
    teams_this_year_df = teams_df[(teams_df['firstYear'] <= thisyear) & (teams_df['lastYear'] >= thisyear)]
    for oneindex, oneteam in teams_this_year_df.iterrows():
        oneteamname = oneteam[0]
        itsleague = oneteam[1][0:1]
        afile = '{:.0f}'.format(thisyear)+oneteamname+'.EV'+itsleague
        if (afile in (os.listdir('retrosheet/'))):
            allfiles.append('retrosheet/'+afile)
    print('files identified')
    
    for filename in allfiles:
#        print('Parsing file: ',filename)
        for line in io.open(filename,encoding='utf-8').readlines():
            linearray = line.split(',')
#            print('BEFORE: ',linearray, '\n')
            linearray[-1] = linearray[-1][0:-1]  # cut off newline character
#            print('AFTER chopping of newline: ',linearray, '\n')
            if (linearray[0] == 'id'):
                # 'id' starts the next game, so immediately advance gamecount  
                # record prior game data by adding it to the gamelogs list, unless gamecount = 0 in which case no data yet
                if (gamecount >= 0):
                    thisgamedict['info'] = info_dict
                    thisgamedict['visiting_starters'] = vis_start_list.tolist()
                    thisgamedict['home_starters'] = home_start_list.tolist()
                    thisgamedict['events'] = events.tolist()
                    thisgamedict['data'] = data_list.tolist()
                    # append the dictionary describing this game to the list of games
                    allgames.append(thisgamedict)

                # advance the gamecount and initialize a new game
                gamecount = gamecount + 1
#                print('Parsing game number {0:,.0f} (0-based counting)'.format(gamecount))
                thisgamedict = {}
                info_dict = {}
                start_dict = {}
                vis_start_list = np.empty((0,4),dtype=object)
                home_start_list = np.empty((0,4),dtype=object)
                vis_sub_list = np.empty((0,4),dtype=object)
                home_sub_list = np.empty((0,4),dtype=object)
                events = np.empty((0,12),dtype=object)
                data_list = np.empty((0,2),dtype=object)
                eventnum = 1

                # the first thing to go into the new game is the game id.
                # the [0:-1] is there to cut off the newline character
                thisgamedict['id'] = linearray[1]

            elif (linearray[0] == 'version'):
                thisgamedict['version'] = linearray[1]
            elif (linearray[0] == 'info'):
                info_dict[linearray[1]] = linearray[2]
            elif (linearray[0] == 'start'):
                thisbatter = np.empty((1,4),dtype=object)
                if (linearray[3] == '0'):
                    thisbatter[0,0] = linearray[1]
                    thisbatter[0,1] = linearray[2]
                    thisbatter[0,2] = linearray[4]
                    thisbatter[0,3] = linearray[5]
                    vis_start_list = np.vstack((vis_start_list, thisbatter))
                elif (linearray[3] == '1'):
                    thisbatter[0,0] = linearray[1]
                    thisbatter[0,1] = linearray[2]
                    thisbatter[0,2] = linearray[4]
                    thisbatter[0,3] = linearray[5]
                    home_start_list = np.vstack((home_start_list, thisbatter))
                else:
                    print('NEITHER VISITOR NOR HOME????')

            elif (linearray[0] == 'data'):
                thisdata = np.empty((2),dtype=object)
                thisdata[0] = linearray[2]
                thisdata[1] = linearray[3]
                data_list = np.vstack((data_list,thisdata))
            else:
    #            print(linearray)
                thisevent = []
                thisevent.append(eventnum)
                thisevent.append(linearray[0])  # say what type of event: play/com/sub/etc.
                if (linearray[0] == 'play'):
                    for i in range(1,7):
                        thisevent.append(linearray[i])
                    for i in range(8,12):   # comment, sub_batting_order, sub_position empty
                        thisevent.append('')
                elif (linearray[0] == 'com'):
                    for i in range(1,7):
                        thisevent.append('')
                    thisevent.append(linearray[1])
                    for i in range(9,12):
                        thisevent.append('')
                elif (linearray[0] == 'sub'):
                    thisevent.append('')    # inning, fill in later
                    thisevent.append('')    # batting team (vis/home), fill in later
                    thisevent.append(linearray[1])  # playerID of player coming in
                    for i in range(4,8):
                        thisevent.append('')  # count/pitches/result/comment all blank
                    thisevent.append(linearray[3]) # batting or fielding sub
                    thisevent.append(linearray[4]) # sub's batting order
                    thisevent.append(linearray[5]) # sub's fielding position
                elif (linearray[0] == 'badj'):
                    badj_games.append(thisgamedict['id'])
                    thisevent.append('')    # inning, fill in later
                    thisevent.append('')  # visitor/home, fill in later
                    thisevent.append(linearray[1])  # playerID of adjusting batter                
                    for i in range(4,7):
                        thisevent.append('')  # count/pitches/result/comment all blank
                    thisevent.append(linearray[0]+": "+linearray[2]) # comment should say: "badj: {hand adjusted to}"
                    for i in range(8,11):
                        thisevent.append('')    # sub-related variables are empty
                elif (linearray[0] == 'padj'):
                    padj_games.append(thisgamedict['id'])
                    thisevent.append('')    # inning, fill in later
                    thisevent.append('')  # visitor/home, fill in later
                    thisevent.append(linearray[1])  # playerID of adjusting batter                
                    for i in range(4,7):
                        thisevent.append('')  # count/pitches/result/comment all blank
                    thisevent.append(linearray[0]+": "+linearray[2]) # comment should say: "padj: {hand adjusted to}"
                    for i in range(8,11):
                        thisevent.append('')    # sub-related variables are empty
                elif (linearray[0] == 'ladj'):
                    ladj_games.append(thisgamedict['id'])
                    thisevent.append('')    # inning, fill in later
                    thisevent.append(linearray[1])  # visitor/home
                    for i in range(3,7):
                        thisevent.append('')  # playerID/count/pitches/result/comment all blank
                    thisevent.append(linearray[0]+": "+linearray[2]) # comment should say: "ladj: {whatever}"
                    thisevent.append(linearray[1])  # visitor/home - team making adjustment
                    for i in range(9,11):
                        thisevent.append('')    # sub-related variables are empty
                else:
                    print(linearray)
                events = np.vstack((events,thisevent))
                eventnum = eventnum + 1

    # add data for the final game, which we missed above
    thisgamedict['info'] = info_dict
    thisgamedict['visiting_starters'] = vis_start_list.tolist()
    thisgamedict['home_starters'] = home_start_list.tolist()
    thisgamedict['events'] = events.tolist()
    thisgamedict['data'] = data_list.tolist()
    allgames.append(thisgamedict)

    print('{0:.0f}: processed {1:,.0f} games!'.format(thisyear,len(allgames)))

    outfilename = 'raw/events_raw_{:.0f}.json'.format(thisyear)
    with open(outfilename, 'w') as f:
        json.dump(allgames, f)
    print('File written:',outfilename)
    print('\n')

In [ ]:
ldf = pandas.DataFrame(ladj_games)
bdf = pandas.DataFrame(badj_games)
pdf = pandas.DataFrame(padj_games)

ldf.columns = ['gameID']
bdf.columns = ['gameID']
pdf.columns = ['gameID']

pdf